In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
ls

drive/  sample_data/


In [4]:
cd drive/MyDrive/

/content/drive/MyDrive


In [5]:
cd DIC


/content/drive/MyDrive/DIC


In [6]:
import cv2
import numpy as np

# Load images (grayscale)
reference_image = cv2.imread('DSLRImages/refrence.png', cv2.IMREAD_GRAYSCALE)
deformed_image = cv2.imread('DSLRImages/deformed.png', cv2.IMREAD_GRAYSCALE)

# Define subset size and search window size
subset_size = 21  # 21x21 pixels
search_window_size = 10  # +/- 10 pixels

# Function to compute ZNCC
def zncc(subset, search_area):
    subset_mean = np.mean(subset)
    search_area_mean = np.mean(search_area)
    numerator = np.sum((subset - subset_mean) * (search_area - search_area_mean))
    denominator = np.sqrt(np.sum((subset - subset_mean) ** 2) * np.sum((search_area - search_area_mean) ** 2))
    return numerator / denominator

# Initialize displacement fields
displacement_x = np.zeros(reference_image.shape)
displacement_y = np.zeros(reference_image.shape)

# Loop through each point in the reference image grid
for y in range(search_window_size, reference_image.shape[0] - search_window_size - subset_size):
    for x in range(search_window_size, reference_image.shape[1] - search_window_size - subset_size):
        # Extract subset from reference image
        subset = reference_image[y:y+subset_size, x:x+subset_size]

        # Initialize maximum ZNCC value
        max_zncc = -1
        best_dx = 0
        best_dy = 0

        # Search in the deformed image
        for dy in range(-search_window_size, search_window_size + 1):
            for dx in range(-search_window_size, search_window_size + 1):
                # Extract search area from deformed image
                search_area = deformed_image[y+dy:y+dy+subset_size, x+dx:x+dx+subset_size]

                # Compute ZNCC
                zncc_value = zncc(subset, search_area)

                # Update best match
                if zncc_value > max_zncc:
                    max_zncc = zncc_value
                    best_dx = dx
                    best_dy = dy

        # Record displacement
        displacement_x[y, x] = best_dx
        displacement_y[y, x] = best_dy

# Visualize the displacement field
plt.figure(figsize=(10, 8))
plt.quiver(displacement_x, displacement_y)
plt.title('Displacement Field')
plt.show()


KeyboardInterrupt: 

In [ ]:
import cv2
import numpy as np
import multiprocessing as mp
from functools import partial

# Load images (grayscale)
reference_image = cv2.imread('DSLRImages/refrence.png', cv2.IMREAD_GRAYSCALE)
deformed_image = cv2.imread('DSLRImages/deformed.png', cv2.IMREAD_GRAYSCALE)

# Define subset size and search window size
subset_size = 21  # 21x21 pixels
search_window_size = 10  # +/- 10 pixels

# Function to compute ZNCC
def zncc(subset, search_area):
    subset_mean = np.mean(subset)
    search_area_mean = np.mean(search_area)
    numerator = np.sum((subset - subset_mean) * (search_area - search_area_mean))
    denominator = np.sqrt(np.sum((subset - subset_mean) ** 2) * np.sum((search_area - search_area_mean) ** 2))
    return numerator / denominator

# Function to process each grid point
def process_point(y, x, reference_image, deformed_image, subset_size, search_window_size):
    subset = reference_image[y:y+subset_size, x:x+subset_size]

    max_zncc = -1
    best_dx = 0
    best_dy = 0

    for dy in range(-search_window_size, search_window_size + 1):
        for dx in range(-search_window_size, search_window_size + 1):
            search_area = deformed_image[y+dy:y+dy+subset_size, x+dx:x+dx+subset_size]
            zncc_value = zncc(subset, search_area)
            if zncc_value > max_zncc:
                max_zncc = zncc_value
                best_dx = dx
                best_dy = dy

    return y, x, best_dx, best_dy

# Initialize displacement fields
displacement_x = np.zeros(reference_image.shape)
displacement_y = np.zeros(reference_image.shape)

# Create a list of grid points to process
points = [(y, x) for y in range(search_window_size, reference_image.shape[0] - search_window_size - subset_size)
                  for x in range(search_window_size, reference_image.shape[1] - search_window_size - subset_size)]

# Use multiprocessing to process the points in parallel
pool = mp.Pool(mp.cpu_count())

# Partial function to pass fixed arguments
partial_process_point = partial(process_point, reference_image=reference_image, deformed_image=deformed_image,
                                subset_size=subset_size, search_window_size=search_window_size)

# Process points in parallel
results = pool.starmap(partial_process_point, points)

# Close the pool
pool.close()
pool.join()

# Populate the displacement fields with results
for y, x, best_dx, best_dy in results:
    displacement_x[y, x] = best_dx
    displacement_y[y, x] = best_dy

# Visualize the displacement field
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 8))
plt.quiver(displacement_x, displacement_y)
plt.title('Displacement Field')
plt.show()


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
import multiprocessing as mp
print(mp.cpu_count())

8


In [ ]:
import cupy as cp
import cv2
import matplotlib.pyplot as plt

# Load images (grayscale)
reference_image = cv2.imread('DSLRImages/refrence.png', cv2.IMREAD_GRAYSCALE)
deformed_image = cv2.imread('DSLRImages/deformed.png', cv2.IMREAD_GRAYSCALE)

# Convert images to GPU arrays using CuPy
reference_image_gpu = cp.array(reference_image)
deformed_image_gpu = cp.array(deformed_image)

# Define subset size and search window size
subset_size = 21  # 21x21 pixels
search_window_size = 10  # +/- 10 pixels

# Function to compute ZNCC on GPU
def zncc_gpu(subset, search_area):
    subset_mean = cp.mean(subset)
    search_area_mean = cp.mean(search_area)
    numerator = cp.sum((subset - subset_mean) * (search_area - search_area_mean))
    denominator = cp.sqrt(cp.sum((subset - subset_mean) ** 2) * cp.sum((search_area - search_area_mean) ** 2))
    return numerator / denominator

# Initialize displacement fields on the GPU
displacement_x_gpu = cp.zeros(reference_image_gpu.shape)
displacement_y_gpu = cp.zeros(reference_image_gpu.shape)

# Loop through each point in the reference image grid
for y in range(search_window_size, reference_image_gpu.shape[0] - search_window_size - subset_size):
    for x in range(search_window_size, reference_image_gpu.shape[1] - search_window_size - subset_size):
        # Extract subset from reference image
        subset = reference_image_gpu[y:y+subset_size, x:x+subset_size]

        # Initialize maximum ZNCC value
        max_zncc = -1
        best_dx = 0
        best_dy = 0

        # Search in the deformed image
        for dy in range(-search_window_size, search_window_size + 1):
            for dx in range(-search_window_size, search_window_size + 1):
                # Extract search area from deformed image
                search_area = deformed_image_gpu[y+dy:y+dy+subset_size, x+dx:x+dx+subset_size]

                # Compute ZNCC on GPU
                zncc_value = zncc_gpu(subset, search_area)

                # Update best match
                if zncc_value > max_zncc:
                    max_zncc = zncc_value
                    best_dx = dx
                    best_dy = dy

        # Record displacement
        displacement_x_gpu[y, x] = best_dx
        displacement_y_gpu[y, x] = best_dy

# Transfer the results back to CPU for visualization
displacement_x = cp.asnumpy(displacement_x_gpu)
displacement_y = cp.asnumpy(displacement_y_gpu)

# Visualize the displacement field
plt.figure(figsize=(10, 8))
plt.quiver(displacement_x, displacement_y)
plt.title('Displacement Field')
plt.show()


KeyboardInterrupt: 

In [7]:
import torch

if torch.cuda.is_available():
    print(f"CUDA is available. Number of GPUs: {torch.cuda.device_count()}")
    print(f"Current GPU: {torch.cuda.get_device_name(torch.cuda.current_device())}")
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"GPU Memory Usage: {torch.cuda.memory_allocated()} bytes allocated, {torch.cuda.memory_reserved()} bytes reserved")
else:
    print("CUDA is not available. Using CPU instead.")



CUDA is available. Number of GPUs: 1
Current GPU: Tesla T4
CUDA Version: 12.1
GPU Memory Usage: 0 bytes allocated, 0 bytes reserved


In [ ]:
import cupy as cp
import cv2
import matplotlib.pyplot as plt

# Load images (grayscale)
reference_image = cv2.imread('DSLRImages/refrence.png', cv2.IMREAD_GRAYSCALE)
deformed_image = cv2.imread('DSLRImages/deformed.png', cv2.IMREAD_GRAYSCALE)

# Convert images to GPU arrays using CuPy
reference_image_gpu = cp.array(reference_image)
deformed_image_gpu = cp.array(deformed_image)

# Verify GPU allocation
print(f"Reference image on device: {reference_image_gpu.device}")
print(f"Deformed image on device: {deformed_image_gpu.device}")

# Define subset size and search window size
subset_size = 21  # 21x21 pixels
search_window_size = 10  # +/- 10 pixels

# Function to compute ZNCC on GPU
def zncc_gpu(subset, search_area):
    subset_mean = cp.mean(subset)
    search_area_mean = cp.mean(search_area)
    numerator = cp.sum((subset - subset_mean) * (search_area - search_area_mean))
    denominator = cp.sqrt(cp.sum((subset - subset_mean) ** 2) * cp.sum((search_area - search_area_mean) ** 2))
    return numerator / denominator

# Initialize displacement fields on the GPU
displacement_x_gpu = cp.zeros(reference_image_gpu.shape)
displacement_y_gpu = cp.zeros(reference_image_gpu.shape)

# Loop through each point in the reference image grid
for y in range(search_window_size, reference_image_gpu.shape[0] - search_window_size - subset_size):
    for x in range(search_window_size, reference_image_gpu.shape[1] - search_window_size - subset_size):
        # Extract subset from reference image
        subset = reference_image_gpu[y:y+subset_size, x:x+subset_size]

        # Initialize maximum ZNCC value
        max_zncc = -1
        best_dx = 0
        best_dy = 0

        # Search in the deformed image
        for dy in range(-search_window_size, search_window_size + 1):
            for dx in range(-search_window_size, search_window_size + 1):
                # Extract search area from deformed image
                search_area = deformed_image_gpu[y+dy:y+dy+subset_size, x+dx:x+dx+subset_size]

                # Compute ZNCC on GPU
                zncc_value = zncc_gpu(subset, search_area)

                # Update best match
                if zncc_value > max_zncc:
                    max_zncc = zncc_value
                    best_dx = dx
                    best_dy = dy

        # Record displacement
        displacement_x_gpu[y, x] = best_dx
        displacement_y_gpu[y, x] = best_dy

        # Synchronize to ensure all operations are complete
        cp.cuda.Stream.null.synchronize()

# Transfer the results back to CPU for visualization
displacement_x = cp.asnumpy(displacement_x_gpu)
displacement_y = cp.asnumpy(displacement_y_gpu)

# Visualize the displacement field
plt.figure(figsize=(10, 8))
plt.quiver(displacement_x, displacement_y)
plt.title('Displacement Field')
plt.show()


Reference image on device: <CUDA Device 0>
Deformed image on device: <CUDA Device 0>
